In [7]:
import requests
import pandas as pd
idx = pd.IndexSlice
from bs4 import BeautifulSoup
from tqdm import tqdm

## 1. State data
## 1.1 Import one file

In [2]:
url = 'https://www.census.gov/construction/bps/xls/msamonthly_202111.xls'
df = pd.read_excel(url, header = 7)
df.head(3)

,CSA,CBSA,Name,Total,1 Unit,2 Units,3 and 4 Units,5 Units or More,Num of Structures With 5 Units or More,Monthly Coverage Percent*,Unnamed: 10,Total.1,1 Unit.1,2 Units.1,3 and 4 Units.1,5 Units or More.1,Num of Structures With 5 Units or More.1
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,999.0,10180.0,"Abilene, TX ...",43.0,35.0,8.0,0.0,0.0,0.0,96.0,NaN,827.0,456.0,82.0,0.0,289.0,7.0
2,184.0,10420.0,"Akron, OH ...",61.0,51.0,0.0,4.0,6.0,1.0,61.0,NaN,735.0,560.0,10.0,99.0,66.0,13.0


## 1.2 Import iteratively

### 1.2.1 Scrape all xls files

https://www.census.gov/construction/bps/statemonthly.html

In [8]:
short_url = 'https://www.census.gov/'
url = 'https://www.census.gov/construction/bps/statemonthly.html'
response = requests.get(url, timeout=10)
soup = BeautifulSoup(response.content, 'html.parser')
urls = []
tags = soup('a')
for tag in tqdm(tags):
    href = (tag.get('href', None))
    if type(href)==str:
        if href.endswith(('xls', 'xlsx')):
            csv_url = short_url+href
            urls.append(csv_url)
urls[:3]

100%|██████████| 70/70 [00:00<00:00, 219432.94it/s]


['https://www.census.gov//construction/bps/xls/statemonthly_202101.xls',
 'https://www.census.gov//construction/bps/xls/statemonthly_202102.xls',
 'https://www.census.gov//construction/bps/xls/statemonthly_202103.xls']

### 1.2.2 Download

In [9]:
dfs = []
for url in tqdm(urls):
    date = url[-10:-6] +'-' + url[-6:-4]
    date = pd.to_datetime(date)
    df = pd.read_excel(url, header = 7)
    df['date'] = date
    df = df.iloc[1:,:]
    df.set_index('date',inplace=True)
    dfs.append(df)
df_s = pd.concat(dfs, axis=0)
df_s.head(2)

100%|██████████| 26/26 [00:16<00:00,  1.61it/s]


,Unnamed: 0,Total,1 Unit,2 Units,3 and 4 Units,5 Units or More,Num of Structures With 5 Units or More,Unnamed: 7,Total.1,1 Unit.1,2 Units.1,3 and 4 Units.1,5 Units or More.1,Num of Structures With 5 Units or More.1
date,,,,,,,,,,,,,,
2021-01-01,United States,128837.0,83921.0,2050.0,1581.0,41285.0,1544.0,NaN,128837.0,83921.0,2050.0,1581.0,41285.0,1544.0
2021-01-01,Northeast Region,13449.0,4978.0,352.0,237.0,7882.0,324.0,NaN,13449.0,4978.0,352.0,237.0,7882.0,324.0


### 1.2.3 Rename and manipulate data

In [10]:
df_s.rename(columns = {'Unnamed: 0':'Location'}, inplace=True)
df_s.Location = [str(x).strip() for x in df_s.Location]
df_s = df_s.reset_index().set_index(['Location','date'])
df_s = df_s.sort_index(axis=0, level='Location')
df_s.head(3)

Total  1 Unit  2 Units  3 and 4 Units  5 Units or More  \
Location date                                                                  
Alabama  2019-11-01  1218.0  1098.0      8.0           13.0             99.0   
         2019-12-01  1366.0  1023.0     14.0           38.0            291.0   
         2020-01-01  1548.0  1303.0     12.0           21.0            212.0   

                     Num of Structures With 5 Units or More  Unnamed: 7  \
Location date                                                             
Alabama  2019-11-01                                     8.0         NaN   
         2019-12-01                                     8.0         NaN   
         2020-01-01                                    11.0         NaN   

                     Total.1  1 Unit.1  2 Units.1  3 and 4 Units.1  \
Location date                                                        
Alabama  2019-11-01  15700.0   13034.0      130.0            315.0   
         2019-12-01  17078.0   14069.0      144.0            353.0   
         2020-01-01   1548.0    1303.0       12.0             21.0   

                     5 Units or More.1  \
Location date                            
Alabama  2019-11-01             2221.0   
         2019-12-01             2512.0   
         2020-01-01              212.0   

                     Num of Structures With 5 Units or More.1  
Location date                                                  
Alabama  2019-11-01                                      85.0  
         2019-12-01                                      93.0  
         2020-01-01                                      11.0

## 1.3 California data

### 1.3.1 Apartments

In [99]:
ca = df_s.loc[idx['California',:], 'Num of Structures With 5 Units or More']
ca.index = ca.index.droplevel(0)
ca.resample('A').sum()

date
2019-12-31     293.0
2020-12-31    1468.0
2021-12-31    1807.0
Freq: A-DEC, Name: Num of Structures With 5 Units or More, dtype: float64

### 1.3.2 One-unit SFH

In [11]:
ca = df_s.loc[idx['California',:], '1 Unit']
ca.index = ca.index.droplevel(0)
ca.resample('A').sum()

date
2019-12-31    10359.0
2020-12-31    60560.0
2021-12-31    65970.0
Freq: A-DEC, Name: 1 Unit, dtype: float64

# 2. City Data
- https://www.census.gov/construction/bps/msamonthly.html

## 2.1 Scrape all xls files

In [12]:
short_url = 'https://www.census.gov/'
url = 'https://www.census.gov/construction/bps/msamonthly.html'
response = requests.get(url, timeout=10)
soup = BeautifulSoup(response.content, 'html.parser')
urls = []
tags = soup('a')
for tag in tags:
    href = (tag.get('href', None))
    if type(href)==str:
        if href.endswith(('xls', 'xlsx')):
            csv_url = short_url+href
            urls.append(csv_url)
urls[:3]

['https://www.census.gov//construction/bps/xls/msamonthly_202101.xls',
 'https://www.census.gov//construction/bps/xls/msamonthly_202102.xls',
 'https://www.census.gov//construction/bps/xls/msamonthly_202103.xls']

## 2.2 Import data

In [14]:
dfs = []
for url in tqdm(urls):
    date = url[-10:-6] +'-' + url[-6:-4]
    date = pd.to_datetime(date)
    df = pd.read_excel(url, header = 7)
    df['date'] = date
    df = df.iloc[1:,:]
    df.set_index('date',inplace=True)
    dfs.append(df)
df_m = pd.concat(dfs, axis=0)


100%|██████████| 26/26 [00:20<00:00,  1.27it/s]


## 2.3 Manipulate

In [15]:
df_m.rename(columns = {'Name':'Location'}, inplace=True)
df_m.Location = [str(x).strip() for x in df_m.Location]
df_m = df_m.reset_index().set_index(['Location','date'])
df_m = df_m.sort_index(axis=0, level='Location')
df_m.head(3)

CSA     CBSA  Total  1 Unit  2 Units  3 and 4 Units  \
Location    date                                                                
Abilene, TX 2019-11-01  999.0  10180.0   23.0    21.0      2.0            0.0   
            2019-12-01  999.0  10180.0   37.0    31.0      6.0            0.0   
            2020-01-01  999.0  10180.0   29.0    29.0      0.0            0.0   

                        5 Units or More  \
Location    date                          
Abilene, TX 2019-11-01              0.0   
            2019-12-01              0.0   
            2020-01-01              0.0   

                        Num of Structures With 5 Units or More  \
Location    date                                                 
Abilene, TX 2019-11-01                                     0.0   
            2019-12-01                                     0.0   
            2020-01-01                                     0.0   

                        Monthly Coverage Percent*  Unnamed: 10  Total.1  \
Location    date                                                          
Abilene, TX 2019-11-01                       92.0          NaN    313.0   
            2019-12-01                       92.0          NaN    350.0   
            2020-01-01                       92.0          NaN     29.0   

                        1 Unit.1  2 Units.1  3 and 4 Units.1  \
Location    date                                               
Abilene, TX 2019-11-01     303.0       10.0              0.0   
            2019-12-01     334.0       16.0              0.0   
            2020-01-01      29.0        0.0              0.0   

                        5 Units or More.1  \
Location    date                            
Abilene, TX 2019-11-01                0.0   
            2019-12-01                0.0   
            2020-01-01                0.0   

                        Num of Structures With 5 Units or More.1  
Location    date                                                  
Abilene, TX 2019-11-01                                       0.0  
            2019-12-01                                       0.0  
            2020-01-01                                       0.0

## 2.4 Show LA

In [104]:
df_m.columns

Index(['CSA', 'CBSA', 'Total', '1 Unit', '2 Units', '3 and 4 Units',
       '5 Units or More', 'Num of Structures With 5 Units or More',
       'Monthly Coverage Percent*', 'Unnamed: 10', 'Total.1', '1 Unit.1',
       '2 Units.1', '3 and 4 Units.1', '5 Units or More.1',
       'Num of Structures With 5 Units or More.1'],
      dtype='object')

### 2.4.1 Apartments

In [108]:
la = df_m.loc[idx['Los Angeles-Long Beach-Anaheim, CA',:], 'Num of Structures With 5 Units or More']
la.index = la.index.droplevel(0)
la['2020':].resample('A').sum()

date
2020-12-31    470.0
2021-12-31    508.0
Freq: A-DEC, Name: Num of Structures With 5 Units or More, dtype: float64

### 2.4.2 SFH

In [16]:
la = df_m.loc[idx['Los Angeles-Long Beach-Anaheim, CA',:], '1 Unit']
la.index = la.index.droplevel(0)
la['2020':].resample('A').sum()

date
2020-12-31     9437.0
2021-12-31    10925.0
Freq: A-DEC, Name: 1 Unit, dtype: float64